In [1]:
import tensorflow as tf

# CNN + LSTM network

In [2]:
def conv2d_layer(prev_layer, num_filters, kernel, strides):
    return tf.layers.conv2d(prev_layer, num_filters, kernel, strides=strides, activation=tf.nn.relu,
                            kernel_initializer=tf.initializers.truncated_normal, bias_initializer=tf.initializers.constant(0.1))

In [3]:
def lstm_layers(prev_layer, lstm_size):
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size, activation=tf.nn.tanh) for size in [lstm_size, lstm_size]]
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
    
    return tf.nn.dynamic_rnn(cell=multi_rnn_cell, inputs=prev_layer, dtype=tf.float32)

In [4]:
def dropout_layer(prev_layer, hold_prob):
    return tf.nn.dropout(prev_layer, keep_prob=hold_prob)

In [5]:
num_sensors = 113
num_classes = 18
window_size = 24
conv_output = 8
num_filters = 64
lstm_size = 128
kernel_size = (5,1)
strides = 1

In [6]:
X = tf.placeholder(tf.float32, shape=[None, window_size, num_sensors, 1])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes])
hold = tf.placeholder(tf.float32)

In [7]:
conv_1 = conv2d_layer(X, num_filters, kernel_size, strides)
conv_2 = conv2d_layer(conv_1, num_filters, kernel_size, strides)
conv_3 = conv2d_layer(conv_2, num_filters, kernel_size, strides)
conv_4 = conv2d_layer(conv_3, num_filters, kernel_size, strides)

In [8]:
drop_1 = dropout_layer(conv_4, hold)
drop_1.shape

TensorShape([Dimension(None), Dimension(8), Dimension(113), Dimension(64)])

In [9]:
reshaped = tf.reshape(drop_1, [-1, conv_output, num_filters*num_sensors])
reshaped.shape

TensorShape([Dimension(None), Dimension(8), Dimension(7232)])

In [10]:
lstm_output, states = lstm_layers(reshaped, lstm_size)

In [12]:
last_output = lstm_output[:,-1,:]
last_output.shape

TensorShape([Dimension(None), Dimension(128)])

In [13]:
drop_2 = dropout_layer(last_output, hold)
drop_2.shape

TensorShape([Dimension(None), Dimension(128)])

In [14]:
y_pred = tf.layers.dense(drop_2, num_classes, kernel_initializer=tf.initializers.truncated_normal(stddev=0.1),
                          bias_initializer=tf.initializers.constant(0.1))

In [16]:
# cost function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

In [17]:
optimizer = tf.train.RMSPropOptimizer(0.001)
train = optimizer.minimize(cross_entropy)